In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from gurobipy import *

## API Call + Data Preprocessing

In [5]:
# Create a list of symbols
symbols = ["NVDA", "GOOG", "AMZN", "AAPL", "META", "TSLA"]

daily_prices = yf.download(
  tickers = ' '.join(symbols), 
  start = datetime(2023, 12, 29),
  end = datetime(2025, 1, 1)
)['Adj Close']

daily_prices.columns = symbols

# Compute daily simple returns
daily_returns = (
  daily_prices.pct_change()
            .dropna(
              # Drop the first row since we have NaN's
              axis = 0,
              how = 'any',
              inplace = False
              )
)

# daily_returns = daily_returns.reset_index()
daily_returns

[*********************100%%**********************]  6 of 6 completed


,NVDA,GOOG,AMZN,AAPL,META,TSLA
Date,,,,,,
2024-01-02,-0.035787,-0.013229,-0.009721,-0.021669,-0.027341,-0.000241
2024-01-03,-0.007488,-0.009738,0.005732,-0.005256,-0.012436,-0.040134
2024-01-04,-0.012700,-0.026268,-0.016529,0.007693,0.009018,-0.002181
2024-01-05,-0.004013,0.004634,-0.004709,0.013915,0.022897,-0.001849
2024-01-08,0.024175,0.026577,0.022855,0.019065,0.064281,0.012464
...,...,...,...,...,...,...
2024-12-24,0.011478,0.017729,0.008062,0.013170,0.003938,0.073572
2024-12-26,0.003176,-0.008732,-0.002379,-0.007240,-0.002068,-0.017630
2024-12-27,-0.013242,-0.014534,-0.015525,-0.005867,-0.020868,-0.049479


## Optimizing Portfolio
Optimizing the stock portfolio strategy by using the historical volatility and average daily return to minimize risk while ensuring a daily return of at least 0.25%.

In [ ]:
# Mean returns and covariance matrix
mean_returns_daily = daily_returns.mean()*100 # returns in %, 
cov_matrix = np.cov(daily_returns.T) # np.cov expects rows to represent variables so .T

# Initialize the model
m = Model("Portfolio")

# Create variables (bounded from 0 to 1 for the proportion each stock represents in portfolio)
weights = m.addVars(len(symbols), lb=0, ub=1, name="weights")

# Set the objective: maximize Sharpe ratio (minimize its negative)
m.setObjective(quicksum(
    weights[i] * cov_matrix[i, j] * weights[j] for i in range(len(symbols)) for j in range(len(symbols))
    ), GRB.MINIMIZE)

# Add constraint: weights must sum to 1
m.addConstr(quicksum(weights[i] for i in range(len(symbols))) == 1, name="weights_sum")

# Add constraint: ensure daily return is at least 0.25%
m.addConstr(quicksum(weights[i] * mean_returns_daily[i] for i in range(len(symbols))) >= 0.25, name='daily_return')

# Optimize the model
m.optimize()

# Print results
if m.status == GRB.OPTIMAL:
    print("Optimal Sharpe Ratio Achieved")
    print("Portfolio Weights:")
    for i, symbol in enumerate(symbols):
        print(f"{symbol}: {weights[i].x}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x8218c1c2
Model has 21 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]


  Objective range  [0e+00, 0e+00]
  QObjective range [3e-04, 3e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+00]
Presolve time: 0.02s
Presolved: 2 rows, 6 columns, 12 nonzeros
Presolved model has 21 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 5
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.49128437e+03 -2.49128437e+03  5.00e+03 3.96e-06  2.90e+05     0s
   1   3.52543068e+02 -3.54357001e+02  3.30e+02 2.62e-07  2.20e+04     0s
   2   9.13313289e-03 -2.97105887e+00  4.05e+00 3.21e-09  2.89e+02     0s
   3   3.04242268e-04 -2.63598749e+00  4.05e-06 3.16e-15  1.30e+01     0s
   4   3.04223161e-04 -2.49393587e-03  2.49e-10 1.39e-17  1.38e-02     0s
   5   2.93109697e-04  1.60323390e-04  7.38e-12 3.47e-18  6.54e-04     0s


C:\Users\Sterling Hayden\AppData\Local\Temp\ipykernel_19008\116865008.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m.addConstr(quicksum(weights[i] * mean_returns_daily[i] for i in range(len(symbols))) >= 0.25, name='daily_return')


## Efficient Frontier

In [34]:
# Initialize an array of target returns
target = np.linspace(
  start = 0.15, 
  stop = 0.35,
  num = 21
)

efficient_frontier  = [] # target return, risk, and list of weights
for target in target:
  # Create a new model for each target
  m = Model("EfficientFrontier")

  # Create variables (bounded from 0 to 1 for the proportion each stock represents in portfolio)
  weights = m.addVars(len(symbols), lb=0, ub=1, name="weights")

  # Set the objective: maximize Sharpe ratio (minimize its negative)
  m.setObjective(quicksum(
      weights[i] * cov_matrix[i, j] * weights[j] for i in range(len(symbols)) for j in range(len(symbols))
      ), GRB.MINIMIZE)

  # Add constraint: weights must sum to 1
  m.addConstr(quicksum(weights[i] for i in range(len(symbols))) == 1, name="weights_sum")

  # Add constraint: ensure daily return is at least target%
  m.addConstr(quicksum(weights[i] * mean_returns_daily[i] for i in range(len(symbols))) >= target, name='daily_return')

  # Optimize the model
  m.optimize()

  # Check if the model has an optimal solution
  if m.status == GRB.OPTIMAL:
      # Portfolio risk (standard deviation is the square root of variance)
      portfolio_variance = m.ObjVal
      portfolio_risk = np.sqrt(portfolio_variance)

      # Extract weights
      portfolio_weights = [weights[i].x for i in range(len(symbols))]

      # Store target return, risk, and weights
      efficient_frontier.append((target, portfolio_risk, portfolio_weights))
  else:
      print(f"No optimal solution found for target return {target}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xc930c380
Model has 21 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e-04, 3e-03]


  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
Presolve time: 0.03s
Presolved: 2 rows, 6 columns, 12 nonzeros
Presolved model has 21 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 5
 AA' NZ     : 2.100e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.49128437e+03 -2.49128437e+03  5.00e+03 3.93e-06  2.90e+05     0s
   1   3.52481657e+02 -3.54743032e+02  3.30e+02 2.59e-07  2.20e+04     0s
   2   8.90670315e-03 -3.45019119e+00  4.03e+00 3.17e-09  2.91e+02     0s
   3   2.56615543e-04 -3.05489052e+00  4.03e-06 3.16e-15  1.50e+01     0s
   4   2.56579145e-04 -3.06520909e-03  3.52e-10 1.39e-17  1.64e-02     0s
   5   2.29669400e-04  2.01618126e-05  8.51e-12 1.39e-17  1.03e-03     0s
   6   1.65506521e-04  1.17598283e-04  8.33e-17 1.39e-17  2.36e-04  

C:\Users\Sterling Hayden\AppData\Local\Temp\ipykernel_19008\777416773.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  m.addConstr(quicksum(weights[i] * mean_returns_daily[i] for i in range(len(symbols))) >= target, name='daily_return')


   3   2.60360921e-04 -3.01709739e+00  4.04e-06 3.16e-15  1.49e+01     0s
   4   2.60323198e-04 -3.02122843e-03  3.54e-10 1.39e-17  1.62e-02     0s
   5   2.32931246e-04  2.66744420e-05  8.44e-12 1.39e-17  1.02e-03     0s
   6   1.68059100e-04  1.21934493e-04  2.22e-16 6.94e-18  2.27e-04     0s
   7   1.59358513e-04  1.57139698e-04  6.94e-18 6.94e-18  1.09e-05     0s
   8   1.59173992e-04  1.59115514e-04  1.18e-16 6.94e-18  2.88e-07     0s
   9   1.59140618e-04  1.59133975e-04  2.31e-14 7.94e-18  3.27e-08     0s
  10   1.59135158e-04  1.59134869e-04  6.69e-14 6.94e-18  1.42e-09     0s

Barrier solved model in 10 iterations and 0.06 seconds (0.00 work units)
Optimal objective 1.59135158e-04

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 6 columns and 12 nonzeros

In [35]:
efficient_frontier

[(0.15,
  0.012443397816518504,
  [0.5222237800546017,
   0.1523264287333478,
   0.19060844906839267,
   0.09885390307461284,
   0.0359540934633543,
   3.334560569004646e-05]),
 (0.16,
  0.012614878421646132,
  [0.5047269186140105,
   0.13819100643725385,
   0.1825879041241516,
   0.109842717171909,
   0.06461438769951258,
   3.706595289496885e-05]),
 (0.16999999999999998,
  0.012849365727924346,
  [0.48591801487380465,
   0.12275285609190674,
   0.1745662917580883,
   0.1209728705695437,
   0.09138846439347667,
   0.004401502313279097]),
 (0.18,
  0.013139545024348276,
  [0.4663327142522852,
   0.10654385127235959,
   0.16654404590203548,
   0.13218665212692612,
   0.11704640065419429,
   0.011346335792115119]),
 (0.19,
  0.013481458600394826,
  [0.44674845339897357,
   0.09033595419261883,
   0.15852177714415175,
   0.14340030087852138,
   0.1427058533540414,
   0.018287661031660896]),
 (0.19999999999999998,
  0.013871281397182553,
  [0.4271635226273374,
   0.0741276935714393,
   0.1